In [2]:
from mongo_connection import MongoDriver

/Users/benwyant/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
mongo: MongoDriver = MongoDriver('localhost', 27017, 'restaurants')
mongo.connect()

if mongo.collection_size("restaurants_collection") == 0:
    mongo.insert_data('restaurants_collection', 'data/restaurants.json', clear=False)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'restaurants')


#### Q1) How many McDonald's are in NYC?

In [4]:
print("\n Number of McDonald's in NYC: \n")
mongo.aggregate_query("restaurants_collection", [
                                                {
                                                    "$match": {
                                                                "name":"Mcdonald'S"
                                                                }
                                                },
                                                {"$count":"totalMcDonalds"}
                                                ])


 Number of McDonald's in NYC: 

{'totalMcDonalds': 208}


[{'totalMcDonalds': 208}]

#### Q2) How many restaurants are in each bourough?

In [5]:
print("\n Number of restaurants in each bourough: \n")
mongo.aggregate_query("restaurants_collection", [
                                                {
                                                    '$group': {
                                                        '_id': '$borough',
                                                        'count': {'$sum': 1}
                                                    }
                                                }
                                            ])


 Number of restaurants in each bourough: 

{'_id': 'Queens', 'count': 5656}
{'_id': 'Brooklyn', 'count': 6086}
{'_id': 'Bronx', 'count': 2338}
{'_id': 'Manhattan', 'count': 10259}
{'_id': 'Staten Island', 'count': 969}
{'_id': 'Missing', 'count': 51}


[{'_id': 'Queens', 'count': 5656},
 {'_id': 'Brooklyn', 'count': 6086},
 {'_id': 'Bronx', 'count': 2338},
 {'_id': 'Manhattan', 'count': 10259},
 {'_id': 'Staten Island', 'count': 969},
 {'_id': 'Missing', 'count': 51}]

#### Q3) Which boroughs, sorted by count, have the highest number of Chinese restaurants?

In [6]:
print("\n Boroughs with the highest number of Chinese restaurants --> give the number of Chinese restaurants in each boroughough: \n")
mongo.aggregate_query("restaurants_collection", [
                                                { "$match": { "cuisine": "Chinese" } },
                                                {
                                                    "$group": {
                                                    "_id": "$borough",
                                                    "count": { "$sum": 1 }
                                                    }
                                                },
                                                { "$sort": { "count": -1 } }
                                                ])


 Boroughs with the highest number of Chinese restaurants --> give the number of Chinese restaurants in each boroughough: 

{'_id': 'Brooklyn', 'count': 763}
{'_id': 'Queens', 'count': 728}
{'_id': 'Manhattan', 'count': 510}
{'_id': 'Bronx', 'count': 323}
{'_id': 'Staten Island', 'count': 88}
{'_id': 'Missing', 'count': 6}


[{'_id': 'Brooklyn', 'count': 763},
 {'_id': 'Queens', 'count': 728},
 {'_id': 'Manhattan', 'count': 510},
 {'_id': 'Bronx', 'count': 323},
 {'_id': 'Staten Island', 'count': 88},
 {'_id': 'Missing', 'count': 6}]

#### Q4) What are the 10 restaurants with the highest average score, sorted by average score? (min 5 reviews)

In [8]:
print("\n Top 10 restaurants with the highest average score, sorted by average score (min 5 reviews): \n")
mongo.aggregate_query("restaurants_collection", 
                                                   [{"$match": {"grades":{"$gt": 5}}},
                                                    {"$project": {
                                                        "name": 1,
                                                        "borough": 1,
                                                        "cuisine": 1,
                                                        "grades": 1,
                                                        "avgScore": {"$avg": "$grades.score"}
                                                    }},
                                                    {"$sort": {"avgScore": -1}},
                                                    {"$limit": 10}
                                                ])


 Top 10 restaurants with the highest average score, sorted by average score (min 5 reviews): 



[]

#### Q5) Which restaurants have a zipcode that starts with '10', are Italian or Chinese cuisine, and have been graded 'A' in their latest grade?

In [11]:
print("\n Restaurants which have a zipcode that starts with '10' and they are of either Italian or Chinese cuisine and have been graded 'A' in their latest grade: \n")
mongo.search_query(collection_name="restaurants_collection", qu = {
                                                        "address.zipcode": { "$regex": "^10" },
                                                        "$or": [
                                                            { "cuisine": "Italian" },
                                                            { "cuisine": "Chinese" }
                                                        ],
                                                        "grades.0.grade": "A"
                                                    },
                                            proj = {
                                                "_id": 0,
                                                "name": 1,
                                            })


 Restaurants which have a zipcode that starts with '10' and they are of either Italian or Chinese cuisine and have been graded 'A' in their latest grade: 

{'name': 'Happy Garden'}
{'name': 'Happy Garden'}
{'name': 'Isle Of Capri Resturant'}
{'name': 'Marchis Restaurant'}
{'name': 'Crystal Room'}
{'name': 'Forlinis Restaurant'}
{'name': 'Angelo Of Mulberry St.'}
{'name': "Arturo'S"}
{'name': 'V & T Restaurant'}
{'name': 'Barbetta Restaurant'}


[{'name': 'Happy Garden'},
 {'name': 'Happy Garden'},
 {'name': 'Isle Of Capri Resturant'},
 {'name': 'Marchis Restaurant'},
 {'name': 'Crystal Room'},
 {'name': 'Forlinis Restaurant'},
 {'name': 'Angelo Of Mulberry St.'},
 {'name': "Arturo'S"},
 {'name': 'V & T Restaurant'},
 {'name': 'Barbetta Restaurant'}]

#### Q6) Which restaurants are located in the Bronx borough and have an 'American' cuisine?

In [13]:
print("\n All restaurants that are located in the Bronx borough and have an 'American' cuisine: \n")
mongo.search_query(collection_name="restaurants_collection", qu = {"borough": "Bronx", "cuisine": "American"}, 
                   proj={ "_id": 0,
                          "name": 1,
                          "borough": 1,
                          "cuisine": 1}, lim=5)


 All restaurants that are located in the Bronx borough and have an 'American' cuisine: 

{'borough': 'Bronx', 'cuisine': 'American', 'name': 'Wild Asia'}
{'borough': 'Bronx', 'cuisine': 'American', 'name': 'Manhem Club'}
{'borough': 'Bronx', 'cuisine': 'American', 'name': 'The New Starling Athletic Club Of The Bronx'}
{'borough': 'Bronx', 'cuisine': 'American', 'name': 'Yankee Tavern'}
{'borough': 'Bronx', 'cuisine': 'American', 'name': 'The Punch Bowl'}


[{'borough': 'Bronx', 'cuisine': 'American', 'name': 'Wild Asia'},
 {'borough': 'Bronx', 'cuisine': 'American', 'name': 'Manhem Club'},
 {'borough': 'Bronx',
  'cuisine': 'American',
  'name': 'The New Starling Athletic Club Of The Bronx'},
 {'borough': 'Bronx', 'cuisine': 'American', 'name': 'Yankee Tavern'},
 {'borough': 'Bronx', 'cuisine': 'American', 'name': 'The Punch Bowl'}]

#### Q7) Which restaurants have a 'Pizza' cuisine and a 'B' grade in their latest inspection?

In [14]:
print("\n All restaurants that have a 'Pizza' cuisine and a 'B' grade in their latest inspection: \n")
mongo.search_query(collection_name="restaurants_collection",     qu={
                                                                    "cuisine": "Pizza",
                                                                    "grades.0.grade": "B"
                                                                },
                                                                proj={
                                                                    "_id": 0,
                                                                    "name": 1,
                                                                    "borough": 1,
                                                                    "cuisine": 1,
                                                                    "grades": {
                                                                        "$elemMatch": {"grade": "B"}
                                                                    }
                                                                }, lim=5)


 All restaurants that have a 'Pizza' cuisine and a 'B' grade in their latest inspection: 

{'borough': 'Queens', 'cuisine': 'Pizza', 'name': 'Pizza Town', 'grades': [{'date': {'$date': 1408579200000}, 'grade': 'B', 'score': 15}]}
{'borough': 'Queens', 'cuisine': 'Pizza', 'name': 'Barone Pizza', 'grades': [{'date': {'$date': 1415836800000}, 'grade': 'B', 'score': 16}]}
{'borough': 'Staten Island', 'cuisine': 'Pizza', 'name': "Pizza D'Oro", 'grades': [{'date': {'$date': 1407369600000}, 'grade': 'B', 'score': 21}]}
{'borough': 'Manhattan', 'cuisine': 'Pizza', 'name': 'Sal And Carmine Pizza', 'grades': [{'date': {'$date': 1409097600000}, 'grade': 'B', 'score': 15}]}
{'borough': 'Manhattan', 'cuisine': 'Pizza', 'name': 'Famous Famiglia Pizza', 'grades': [{'date': {'$date': 1408492800000}, 'grade': 'B', 'score': 25}]}


[{'borough': 'Queens',
  'cuisine': 'Pizza',
  'name': 'Pizza Town',
  'grades': [{'date': {'$date': 1408579200000}, 'grade': 'B', 'score': 15}]},
 {'borough': 'Queens',
  'cuisine': 'Pizza',
  'name': 'Barone Pizza',
  'grades': [{'date': {'$date': 1415836800000}, 'grade': 'B', 'score': 16}]},
 {'borough': 'Staten Island',
  'cuisine': 'Pizza',
  'name': "Pizza D'Oro",
  'grades': [{'date': {'$date': 1407369600000}, 'grade': 'B', 'score': 21}]},
 {'borough': 'Manhattan',
  'cuisine': 'Pizza',
  'name': 'Sal And Carmine Pizza',
  'grades': [{'date': {'$date': 1409097600000}, 'grade': 'B', 'score': 15}]},
 {'borough': 'Manhattan',
  'cuisine': 'Pizza',
  'name': 'Famous Famiglia Pizza',
  'grades': [{'date': {'$date': 1408492800000}, 'grade': 'B', 'score': 25}]}]

#### Q8) What are the top 5 restaurants with the highest average score for their grades? Only show their name, cuisine, and average score

In [15]:
print("\n Top 5 restaurants with the highest average score for their grades, and only show their name, cuisine, and average score: \n")
mongo.search_query(collection_name="restaurants_collection",qu={}, proj={"name": 1, "cuisine": 1, "avgScore": {"$avg": "$grades.score"}, "_id": 0}, lim=5)


 Top 5 restaurants with the highest average score for their grades, and only show their name, cuisine, and average score: 

{'cuisine': 'Bakery', 'name': 'Morris Park Bake Shop', 'avgScore': 8.2}
{'cuisine': 'Hamburgers', 'name': "Wendy'S", 'avgScore': 13.75}
{'cuisine': 'Irish', 'name': 'Dj Reynolds Pub And Restaurant', 'avgScore': 9.25}
{'cuisine': 'American', 'name': 'Riviera Caterer', 'avgScore': 9.0}
{'cuisine': 'Jewish/Kosher', 'name': 'Tov Kosher Kitchen', 'avgScore': 17.75}


[{'cuisine': 'Bakery', 'name': 'Morris Park Bake Shop', 'avgScore': 8.2},
 {'cuisine': 'Hamburgers', 'name': "Wendy'S", 'avgScore': 13.75},
 {'cuisine': 'Irish',
  'name': 'Dj Reynolds Pub And Restaurant',
  'avgScore': 9.25},
 {'cuisine': 'American', 'name': 'Riviera Caterer', 'avgScore': 9.0},
 {'cuisine': 'Jewish/Kosher', 'name': 'Tov Kosher Kitchen', 'avgScore': 17.75}]

#### Q9) Which restaurants have a 'Chinese' or 'Japanese' cuisine, are located in the 'Queens' borough, and have a grade 'A' in their latest inspection?

In [16]:
print("\n All restaurants that have a 'Chinese' or 'Japanese' cuisine, and are located in the 'Queens' borough, and have a grade 'A' in their latest inspection. \n")
mongo.search_query(collection_name="restaurants_collection",     qu={
                                                                    "borough": "Queens", 
                                                                    "cuisine": {"$in": ["Chinese", "Japanese"]}, 
                                                                    "grades.0.grade": "A"
                                                                }, 
                                                                proj={
                                                                    "name": 1, 
                                                                    "borough": 1, 
                                                                    "cuisine": 1, 
                                                                    "grades": 1,
                                                                    "_id": 0
                                                                },
                                                                lim=5)


 All restaurants that have a 'Chinese' or 'Japanese' cuisine, and are located in the 'Queens' borough, and have a grade 'A' in their latest inspection. 

{'borough': 'Queens', 'cuisine': 'Chinese', 'grades': [{'date': {'$date': 1398038400000}, 'grade': 'A', 'score': 10}, {'date': {'$date': 1384214400000}, 'grade': 'A', 'score': 5}, {'date': {'$date': 1370304000000}, 'grade': 'A', 'score': 12}, {'date': {'$date': 1352851200000}, 'grade': 'A', 'score': 10}, {'date': {'$date': 1349913600000}, 'grade': 'P', 'score': 0}, {'date': {'$date': 1337817600000}, 'grade': 'A', 'score': 13}, {'date': {'$date': 1323302400000}, 'grade': 'A', 'score': 12}, {'date': {'$date': 1311120000000}, 'grade': 'A', 'score': 11}], 'name': 'Ho Mei Restaurant'}
{'borough': 'Queens', 'cuisine': 'Japanese', 'grades': [{'date': {'$date': 1403654400000}, 'grade': 'A', 'score': 6}, {'date': {'$date': 1389225600000}, 'grade': 'A', 'score': 10}, {'date': {'$date': 1367452800000}, 'grade': 'A', 'score': 2}, {'date': {'$dat

[{'borough': 'Queens',
  'cuisine': 'Chinese',
  'grades': [{'date': {'$date': 1398038400000}, 'grade': 'A', 'score': 10},
   {'date': {'$date': 1384214400000}, 'grade': 'A', 'score': 5},
   {'date': {'$date': 1370304000000}, 'grade': 'A', 'score': 12},
   {'date': {'$date': 1352851200000}, 'grade': 'A', 'score': 10},
   {'date': {'$date': 1349913600000}, 'grade': 'P', 'score': 0},
   {'date': {'$date': 1337817600000}, 'grade': 'A', 'score': 13},
   {'date': {'$date': 1323302400000}, 'grade': 'A', 'score': 12},
   {'date': {'$date': 1311120000000}, 'grade': 'A', 'score': 11}],
  'name': 'Ho Mei Restaurant'},
 {'borough': 'Queens',
  'cuisine': 'Japanese',
  'grades': [{'date': {'$date': 1403654400000}, 'grade': 'A', 'score': 6},
   {'date': {'$date': 1389225600000}, 'grade': 'A', 'score': 10},
   {'date': {'$date': 1367452800000}, 'grade': 'A', 'score': 2},
   {'date': {'$date': 1347321600000}, 'grade': 'A', 'score': 11},
   {'date': {'$date': 1329091200000}, 'grade': 'A', 'score': 4},

#### Q10) What is the distribution of restaurants across different cuisines in the NYC boroughs

In [23]:
print("\n Query to make a visualization of the distribution of restaurants across different cuisines in the NYC boroughs: \n")
res: list = mongo.aggregate_query("restaurants_collection", [{"$group": {"_id": {"borough": "$borough", "cuisine": "$cuisine"}, "count": {"$sum": 1}}},
                                                                {"$project": {"borough": "$_id.borough", "cuisine": "$_id.cuisine", "count": "$count", "_id": 0}}]
                                                                , show=False)
for item in res[:10]:
    print(item)


 Query to make a visualization of the distribution of restaurants across different cuisines in the NYC boroughs: 

{'borough': 'Staten Island', 'cuisine': 'Delicatessen', 'count': 16}
{'borough': 'Bronx', 'cuisine': 'Ice Cream, Gelato, Yogurt, Ices', 'count': 27}
{'borough': 'Queens', 'cuisine': 'Pakistani', 'count': 11}
{'borough': 'Bronx', 'cuisine': 'Chicken', 'count': 108}
{'borough': 'Queens', 'cuisine': 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 'count': 300}
{'borough': 'Brooklyn', 'cuisine': 'Russian', 'count': 65}
{'borough': 'Bronx', 'cuisine': 'Middle Eastern', 'count': 3}
{'borough': 'Staten Island', 'cuisine': 'Turkish', 'count': 1}
{'borough': 'Staten Island', 'cuisine': 'Hamburgers', 'count': 25}
{'borough': 'Queens', 'cuisine': 'Peruvian', 'count': 40}
